# 🚀 Flusso di Esecuzione per Training Qwen su Colab

**NOTA**: Questo notebook è configurato per un repository pubblico su GitHub.
Assicurati che il repository `Attapulgite999/prove` sia pubblico e contenga tutti i file necessari.

### 0. **Verifica Connessione Colab**

In [ ]:
# Verifica connessione internet
!curl -s -o /dev/null -w "%{http_code}" https://www.google.com

# Verifica GPU (senza importare torch per evitare conflitti)
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader,nounits

# Verifica RAM
!free -h

# Verifica CPU
!nproc

### 1. **Preparazione Ambiente Colab**

In [ ]:
# Monta Google Drive (OBBLIGATORIO se vuoi salvare il modello addestrato e il file GGUF)
from google.colab import drive
drive.mount('/content/drive')

# Clona il repository pubblico
!rm -rf prove

!git clone https://github.com/Attapulgite999/prove.git
%cd prove

### 2. **Installazione Dipendenze Base**

In [ ]:
# Aggiorna apt e installa python3 e pip (solitamente già presenti in Colab, ma non fa male)
!apt-get update && apt-get install -y python3 python3-pip

# Installa le dipendenze principali (versioni compatibili con Axolotl)
!pip uninstall -y unsloth unsloth-zoo

# Installa PyTorch 2.6.0 e versioni compatibili per Axolotl
!pip uninstall -y torch torchvision torchaudio triton xformers
!pip install --index-url https://download.pytorch.org/whl/cu126 torch==2.6.0 torchvision==0.23.0 torchaudio==2.8.0 --no-cache-dir

# Installa Axolotl (framework per fine-tuning)
!pip install --no-deps axolotl[deepspeed]==0.12.2
# Installa timm (dipendenza mancante per accelerate)
!pip install timm

# Installa dipendenze aggiuntive per Axolotl
!pip install transformers==4.55.2 datasets==4.0.0 accelerate==1.10.0 peft==0.17.0 trl==0.21.0
# Installa dipendenze mancanti per Axolotl
!pip install colorama addict adlfs art autoawq axolotl-contribs-lgpl==0.0.6 axolotl-contribs-mit==0.0.5 bitsandbytes==0.47.0 evaluate==0.4.1 fire kernels==0.9.0 langdetect==1.0.9 liger-kernel==0.6.1 lm_eval==0.4.7 mistral-common==1.8.3 modal==1.0.2 ocifs==1.3.2 optimum==1.16.2 s3fs schedulefree==1.4.1 trackio xformers==0.0.29.post3


### 3. **Esecuzione Script Training**

In [ ]:
# Esegui il training con Axolotl. Il modello verrà salvato nella directory axolotl_training/outputs/
!accelerate launch -m axolotl.cli.train axolotl_training/config/qwen_axolotl.yaml

### 4. **Scarica il File GGUF (sul tuo computer locale)**

In [ ]:
# Il modello LoRA addestrato viene salvato nella directory axolotl_training/outputs/medical_qwen_axolotl/
# Puoi scaricarlo dal tuo ambiente Colab al tuo computer locale usando questo comando:
from google.colab import files

# Scarica il modello LoRA (directory completa)
!zip -r medical_qwen_axolotl.zip axolotl_training/outputs/medical_qwen_axolotl/
files.download('medical_qwen_axolotl.zip')

## 🔧 Miglioramenti Implementati nello Script `setup_and_train_colab.py`

- **Retry automatico**: 3 tentativi per il caricamento del tokenizer e per il training.
- **Pre-check modello**: Verifica disponibilità del modello prima del training.
- **Gestione errori migliorata**: Log dettagliati per un troubleshooting più semplice.
- **Monitoraggio risorse**: Tracking dell'utilizzo di GPU memory e utilization.
- **Keep-alive**: Previene disconnessioni automatiche di Colab.
- **Resume da checkpoint**: Riprende il training dall'ultimo checkpoint in caso di interruzioni.
- **Download dataset automatico**: Scarica il dataset medico da Hugging Face se non presente localmente.
- **Merge LoRA e Conversione GGUF**: Unisce gli adapter LoRA al modello base e lo converte in formato GGUF per LM Studio.

In [ ]:
!ls -lah axolotl_training/outputs/medical_qwen_axolotl/


In [ ]:
import torch, torchvision, torchaudio
print(torch.__version__, torchvision.__version__, torchaudio.__version__)
print(torch.cuda.is_available(), torch.version.cuda)
print(torch.cuda.get_device_name(0))
x = torch.randn(1024, 1024, device='cuda')
y = torch.mm(x, x)
print(y.shape)
